In [1]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data.data_load import (
    load_tables, 
    load_online_instance, 
    load_distances, 
    upload_ONLINE_static_solution
)
from src.data.solution_load import load_solution_dfs, _include_all_city
from src.utils.filtering import flexible_filter
from src.utils.plotting import plot_metrics_comparison_dynamic
from src.data.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df
from src.config.experimentation_config import *
from src.config.SD_experimentation_config import *
from src.config.config import *

data_path = '../data'

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
dist_method = 'haversine'      # Options: ['precalced', 'haversine']

optimization_obj = 'driver_distance'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
# dist_dict = load_distances(data_path, distance_type, instance, dist_method)

metricas = ['service_count', 'vt_count', 'num_drivers', 'driver_extra_time', 'driver_move_distance']


# Upload results

In [2]:
def upload_simulated_instances():
    
    results = {}

    for n_serv in n_services:
        labors_real_dfs = pd.DataFrame()
        labors_static_dfs = pd.DataFrame()
        labors_dynamic_dfs = pd.DataFrame()
        for scenario in scenarios:
            for seed in seeds:
                instance = f'N{n_serv}/{scenario}/seed_{seed}'
                labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)

                for df in [labors_real_df, labors_static_df, labors_dynamic_df]:
                    df['n_serv'] = n_serv
                    df['scenario'] = scenario
                    df['seed'] = seed
                
                labors_real_df = _include_all_city(labors_real_df)
                labors_static_df = _include_all_city(labors_static_df)
                labors_dynamic_df = _include_all_city(labors_dynamic_df)

                labors_real_dfs = pd.concat([labors_real_dfs, labors_real_df])
                labors_static_dfs = pd.concat([labors_static_dfs, labors_static_df])
                labors_dynamic_dfs = pd.concat([labors_dynamic_dfs, labors_dynamic_df])
        
        results[n_serv] = (labors_real_dfs, labors_static_dfs, labors_dynamic_dfs)

    return results
        
results = upload_simulated_instances()    


# Instance exploration

## Global

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, Dropdown


def plot_topology_reactive(results, n_services_list, scenarios, seeds):
    """
    Creates two reactive dropdowns: n_services and city.
    Whenever the user selects a new value, the plots update.
    """

    # ---- helper: available cities for a given n_services ----
    def available_cities(n):
        df_real = results[n][0]
        if "city" in df_real.columns:
            return sorted(df_real["city"].dropna().unique())
        return []

    # ---- reactive plotting function ----
    def draw(n_services, city):
        df_real, df_static, df_dynamic = results[n_services]

        # ---------------- FILTER BY CITY ----------------
        dfR = df_real[df_real['city'] == city].copy()
        dfS = df_static[df_static['city'] == city].copy()
        dfD = df_dynamic[df_dynamic['city'] == city].copy()

        # ---------------- HISTOGRAM DATA ----------------
        hist_rows = []
        for sc in scenarios:
            sub = dfR[dfR["scenario"] == sc]
            for seed, g in sub.groupby("seed"):
                vt_count = int((g["labor_category"] == "VEHICLE_TRANSPORTATION").sum())
                hist_rows.append({"scenario": sc, "seed": seed, "vt": vt_count})

        hist_df = pd.DataFrame(hist_rows)

        # ---------------- BOX PLOT DATA ----------------
        box_rows = []
        for sc in scenarios:
            df_stat_sc = dfS[dfS["scenario"] == sc]
            df_dyn_sc = dfD[dfD["scenario"] == sc]

            stat_by_seed = df_stat_sc.groupby("seed")["service_id"].nunique()
            dyn_by_seed = df_dyn_sc.groupby("seed")["service_id"].nunique()

            union_seeds = sorted(set(stat_by_seed.index) | set(dyn_by_seed.index))
            for seed in union_seeds:
                n_static = int(stat_by_seed.get(seed, 0))
                n_dynamic = int(dyn_by_seed.get(seed, 0))
                total = n_static + n_dynamic
                prop_static = (n_static / total) if total > 0 else np.nan
                box_rows.append({
                    "scenario": sc,
                    "seed": seed,
                    "prop_static": prop_static
                })

        box_df = pd.DataFrame(box_rows)

        # ---------------- PLOTTING ----------------
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=[f"VT counts: {sc}" for sc in scenarios] +
                           ["Static proportion per scenario"],
            specs=[[{"type": "xy"}]*3,
                   [{"colspan": 3, "type": "xy"}, None, None]],
            vertical_spacing=0.15
        )

        # -- top row histograms --
        bins = 20
        for i, sc in enumerate(scenarios):
            h = hist_df[hist_df.scenario == sc]
            if not h.empty:
                vals, edges = np.histogram(h["vt"].values, bins=bins)
                centers = (edges[:-1] + edges[1:]) / 2
                fig.add_trace(
                    go.Bar(x=centers, y=vals, showlegend=False),
                    row=1, col=i+1
                )
            fig.update_xaxes(title_text="VT per seed", row=1, col=i+1)
            fig.update_yaxes(title_text="Frequency", row=1, col=i+1)

        # -- bottom row boxplot --
        for sc in scenarios:
            fig.add_trace(
                go.Box(
                    y=box_df[box_df.scenario == sc]["prop_static"],
                    name=sc,
                    boxmean="sd"
                ),
                row=2, col=1
            )

        fig.update_layout(
            height=700,
            width=1200,
            title=f"Instance topology  |  N={n_services}  |  City={city}",
            showlegend=False,
            template="plotly_white"
        )

        fig.show()

    # ---- WIDGETS ----
    # Default values
    default_n = n_services_list[0]
    default_city = available_cities(default_n)[0]

    # Use interact to automatically re-run draw() on change
    interact(
        draw,
        n_services=Dropdown(options=n_services_list, value=default_n, description="N services"),
        city=Dropdown(options=available_cities(default_n), value='ALL', description="City")
    )

plot_topology_reactive(
    results=results,
    n_services_list=n_services,
    scenarios=scenarios,
    seeds=seeds
)


interactive(children=(Dropdown(description='N services', options=(900, 1000, 1100, 1200), value=900), Dropdown…

# Result visualization

## Load results

In [13]:
import pickle
import pandas as pd
from pathlib import Path

import os

algorithms = []


algorithms.append({
    "name": "OFFLINE",
    "type": "algorithm",
    "color": "#2980B9",  # 🔵 Royal blue – “best possible solution”
    "visible": True,
})
algorithms.append({
    "name": "INSERT",
    "type": "algorithm",
    "color": "#27AE60",  # 🟢 Vibrant green – “first smart improvement”
    "visible": True,
})
algorithms.append({
    "name": "BUFFER_FIXED",
    "type": "algorithm",
    "color": "#16A085",
    "visible": True,
})
algorithms.append({
    "name": "REACT",
    "type": "algorithm",
    "color": "#E67E22",  # 🟠 Bold orange – “fast adaptation”
    "visible": True,
})
algorithms.append({
    "name": "BUFFER_REACT",
    "type": "algorithm",
    "color": "#9B59B6",  # 🟣 Electric purple – “intelligent batching”
    "visible": True,
})

def load_all_results_explicit(
    root_dir: str,
    n_services_list,
    scenarios_list,
    seeds_list,
    dist_methods_list,
    algorithms_list
) -> pd.DataFrame:
    """
    Clean, explicit loader for result files located in:
    root/N{n_services}/{scenario}/seed_{seed}/{dist_method}/{algorithm}.pkl

    Returns a tidy DataFrame with columns:
    [n_services, scenario, seed, dist_method, algorithm, labors_df, moves_df, metadata]
    """
    root = f'{REPO_PATH}/data/resultados/simulation'
    rows = []
    rows2 = []

    for n_services in n_services_list:
        n_dir = os.path.join(root, f'N{n_services}')

        for scenario in scenarios_list:
            
            scenario_dir = os.path.join(n_dir, scenario)

            for seed in seeds_list:
                seed_dir = os.path.join(scenario_dir, f"seed_{seed}")

                for dist_method in dist_methods_list:
                    dist_dir = os.path.join(seed_dir, dist_method)

                    for algorithm in algorithms_list:
                        pkl_path = os.path.join(dist_dir, f"res_algo_{algorithm}.pkl")

                        # Load pickle
                        try:
                            with open(pkl_path, "rb") as f:
                                labors_df, moves_df, metadata = pickle.load(f)
                        except Exception as e:
                            print(f"❌ Error loading {pkl_path}: {e}")
                            continue

                        labors_df = _include_all_city(labors_df)
                        moves_df = _include_all_city(moves_df)

                        rows.append({
                            "n_services": n_services,
                            "scenario": scenario,
                            "seed": seed,
                            "dist_method": dist_method,
                            "algorithm": algorithm,
                            "labors_df": labors_df,
                            "moves_df": moves_df,
                            "metadata": metadata,
                            "path": str(pkl_path)
                        })

                        rows2.append({
                            "n_services": n_services, "scenario": scenario, "seed": seed, "algorithm": algorithm,
                            "labors_df": labors_df, "moves_df": moves_df
                        })

    runs_df = pd.DataFrame(rows)
    
    return pd.DataFrame(rows), runs_df

algorithms_list = ['OFFLINE', 'INSERT', 'BUFFER_FIXED', 'REACT', 'BUFFER_REACT']

results_df, runs_df = load_all_results_explicit(
    root_dir=f'{data_path}/resultados/',
    n_services_list=[900],
    scenarios_list=scenarios,
    seeds_list=seeds,
    dist_methods_list=['haversine'],
    algorithms_list=algorithms_list
)


## Global results

In [14]:
# plot_results_boxplots.py
import os
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from ipywidgets import interact, widgets

# --- you must have this function available in scope ---
# from src.metrics import compute_metrics_with_moves
# (the user already has compute_metrics_with_moves in their codebase)

def build_metrics_df_for_plots(
    runs_df: pd.DataFrame,
    n_services: int,
    city: str,
    metricas: List[str],
    dist_dict: Dict,
    workday_hours: int = 8,
    agg_fn = np.sum
) -> pd.DataFrame:
    """
    Build a tidy dataframe of metric values ready for plotting.

    Parameters
    ----------
    runs_df : pd.DataFrame
        Tidy table with the following required columns:
          - 'n_services' (int)
          - 'scenario' (str)
          - 'seed' (int)
          - 'algorithm' (str)
          - 'labors_df' (pd.DataFrame)
          - 'moves_df' (pd.DataFrame)
        (If your runs_df stores labors/moves differently, you must first
         transform it into this form.)
    n_services : int
        Which experiment size to plot.
    city : str
        City code or 'ALL' (passed to compute_metrics_with_moves).
    metricas : list[str]
        List of metric names (same as used in compute_metrics_with_moves).
    dist_dict : dict
        Distances dictionary required by compute_metrics_with_moves.
    workday_hours : int
        Workday hours parameter forwarded to metric computation.
    agg_fn : callable
        Aggregation function to reduce daily series into a single scalar per run (default: sum).

    Returns
    -------
    pd.DataFrame
        tidy df with columns:
        ['n_services','scenario','seed','algorithm','metric','value','city']
    """
    required_cols = {"n_services", "scenario", "seed", "algorithm", "labors_df", "moves_df"}
    if not required_cols.issubset(set(runs_df.columns)):
        raise ValueError(f"runs_df must contain columns: {required_cols}")

    # filter by n_services
    subset = runs_df.loc[runs_df["n_services"] == n_services].copy()
    if subset.empty:
        raise ValueError(f"No runs found for n_services={n_services}")

    rows = []
    # iterate runs
    for idx, run in subset.iterrows():
        algo = run["algorithm"]
        scn = run["scenario"]
        seed = run["seed"]
        n_s = run["n_services"]
        labors_df = run["labors_df"]
        moves_df = run["moves_df"]

        # compute metrics using your function (it returns a dataframe indexed by day)
        metrics_df = compute_metrics_with_moves(
            labors_df,
            moves_df,
            fechas=['2026-11-11'],            # None -> function should compute for available dates (if your func requires fechas, adapt)
            dist_dict=dist_dict,
            workday_hours=workday_hours,
            city=city,
            skip_weekends=False,
            assignment_type='algorithm',
            dist_method='haversine'
        )

        # If metrics_df is empty, skip
        if metrics_df is None or metrics_df.empty:
            # produce NaNs for requested metrics
            for m in metricas:
                rows.append({
                    "n_services": n_s, "scenario": scn, "seed": seed, "algorithm": algo,
                    "metric": m, "value": np.nan, "city": city
                })
            continue

        # For each metric, reduce to scalar using agg_fn
        for m in metricas:
            if m not in metrics_df.columns:
                val = np.nan
            else:
                series = pd.to_numeric(metrics_df[m], errors='coerce').dropna()
                val = agg_fn(series) if not series.empty else 0.0
            rows.append({
                "n_services": n_s, "scenario": scn, "seed": seed, "algorithm": algo,
                "metric": m, "value": val, "city": city
            })

    tidy = pd.DataFrame(rows)
    # ensure scenario ordering (if you want a particular order: easy, normal, hard)
    scenario_order = ["easy", "normal", "hard"]
    present = [s for s in scenario_order if s in tidy['scenario'].unique()]
    tidy['scenario'] = pd.Categorical(tidy['scenario'], categories=present + [c for c in tidy['scenario'].unique() if c not in present], ordered=True)
    return tidy


def plot_simulation_metrics_boxplots(
    metrics_tidy_df: pd.DataFrame,
    metricas: List[str],
    algorithms_info: List[Dict[str, str]],
    scenarios: List[str],
    save: bool = False,
    save_dir: Optional[str] = None,
    title_prefix: Optional[str] = None,
):
    """
    Instead of one multi-row figure, this function produces
    ONE FIGURE PER METRIC.

    Each figure shows:
    - x-axis: scenario
    - grouped boxplots: one box per algorithm per scenario
    - shared colors across all figures
    """
    # alg_colors stays the same
    alg_colors = {a['name']: a.get('color', None) for a in algorithms_info}

    # NEW: preserve the exact algorithm order provided
    alg_order = [a['name'] for a in algorithms_info]
    algs_present = [a for a in alg_order if a in metrics_tidy_df['algorithm'].unique()]


    for metric in metricas:
        dfm = metrics_tidy_df[metrics_tidy_df['metric'] == metric].copy()

        # Create an empty figure
        fig = go.Figure()

        # Order scenarios if available
        scenario_order = scenarios if all(s in dfm['scenario'].unique() for s in scenarios) \
                                    else sorted(dfm['scenario'].unique())

        # One trace per algorithm
        for alg in algs_present:
            alg_df = dfm[dfm['algorithm'] == alg]
            if alg_df.empty:
                continue

            fig.add_trace(
                go.Box(
                    x=alg_df['scenario'].astype(str),
                    y=alg_df['value'],
                    name=alg,
                    marker_color=alg_colors.get(alg),
                    boxmean="sd",
                    legendgroup=alg,
                    hovertemplate="scenario=%{x}<br>"
                                  "algorithm=%{fullData.name}<br>"
                                  "value=%{y:.2f}<extra></extra>",
                )
            )

        # Layout
        full_title = f"{title_prefix or ''} — {metric}"
        fig.update_layout(
            title=full_title,
            xaxis_title="Scenario",
            yaxis_title=metric,
            boxmode="group",
            width=1100,
            height=450,
            plot_bgcolor="white",
            legend=dict(
                orientation="h",
                yanchor="bottom", y=1.05,
                xanchor="center", x=0.5
            ),
            margin=dict(t=100, b=60),
        )

        fig.update_xaxes(
            categoryorder="array",
            categoryarray=scenario_order,
            tickangle=-15
        )

        # Save if requested
        if save and save_dir:
            os.makedirs(save_dir, exist_ok=True)
            fname = f"{metric}_boxplot.html"
            out_path = os.path.join(save_dir, fname)
            fig.write_html(out_path, include_plotlyjs="cdn")
            print(f"Saved plot to {out_path}")
        else:
            fig.show()



# ---------------------------
# Interactive wrapper
# ---------------------------
def interactive_metrics_boxplots(
    runs_df: pd.DataFrame,
    n_services_list: List[int],
    scenarios: List[str],
    algorithms_info: List[Dict[str, str]],
    metricas: List[str],
    dist_dict: Dict,
    city_options: List[str] = None,
):
    """
    Build ipywidgets.interact UI to choose n_services and city.
    runs_df must be tidy as described in build_metrics_df_for_plots docstring.
    """

    if city_options is None:
        # infer from runs_df (there may be many cities inside labors_df; present a union)
        city_options = ["ALL"]
        # try to infer cities present in run labors_df objects
        cities_set = set()
        for _, r in runs_df.iterrows():
            try:
                ldf = r['labors_df']
                if 'city' in ldf.columns:
                    cities_set.update(ldf['city'].unique().tolist())
            except Exception:
                continue
        city_options += sorted(list(cities_set))

    n_services_dropdown = widgets.Dropdown(
        options=sorted(n_services_list),
        value=sorted(n_services_list)[0],
        description='n_services:',
        style={"description_width": "initial"}
    )

    city_dropdown = widgets.Dropdown(
        options=city_options,
        value=city_options[0],
        description='City:',
        style={"description_width": "initial"}
    )

    def _update(n_services, city):
        # build metrics table
        tidy = build_metrics_df_for_plots(
            runs_df=runs_df,
            n_services=n_services,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict
        )
        # for title: include counts
        title = f"n_services={n_services} | city={city}"
        plot_simulation_metrics_boxplots(
            metrics_tidy_df=tidy,
            metricas=metricas,
            algorithms_info=algorithms_info,
            scenarios=scenarios,
            title_prefix=title
        )

    interact(_update, n_services=n_services_dropdown, city=city_dropdown)


In [15]:
interactive_metrics_boxplots(
    runs_df,
    [900],
    scenarios,
    algorithms,
    metricas,
    dict(),
    ['ALL'] + valid_cities,
)

interactive(children=(Dropdown(description='n_services:', options=(900,), style=DescriptionStyle(description_w…

## EVALUATING

In [6]:
path = f'{REPO_PATH}/data/instances/simu_inst/N900/easy/seed_0/'
labors_df = pd.read_csv(f'{path}/labors_sim_df.csv')
directorio_hist = pd.read_csv(f'{path}/directorio_hist_df.csv')

path = f'{REPO_PATH}/data/resultados/simulation/N900/easy/seed_0/haversine/res_algo_OFFLINE.pkl'
with open(path, "rb") as f:
    labors_algo_df, moves_algo_df, metadata = pickle.load(f)


# CASE 2: City 149


## Instance

In [7]:
flexible_filter(
    labors_df,
    city=149
).sort_values(['schedule_date', 'created_at'])[['service_id', 'labor_id', 'labor_category', 'created_at', 'schedule_date',  'labor_sequence',  'labor_type']]

,service_id,labor_id,labor_category,created_at,schedule_date,labor_sequence,labor_type
550,254329,350449,VEHICLE_TRANSPORTATION,2026-11-05 13:22:41.542000-05:00,2026-11-11 06:30:00-05:00,0,12.0
361,252562,348500,VEHICLE_TRANSPORTATION,2026-11-10 12:15:35.934000-05:00,2026-11-11 06:30:00-05:00,0,12.0
5,244615,340067,VEHICLE_TRANSPORTATION,2026-10-16 17:11:58.700000-05:00,2026-11-11 07:00:00-05:00,0,12.0
6,244617,340069,VEHICLE_TRANSPORTATION,2026-10-20 17:16:18.327000-05:00,2026-11-11 07:00:00-05:00,0,12.0
21,247795,343399,VEHICLE_TRANSPORTATION,2026-10-30 12:12:16.688000-05:00,2026-11-11 07:00:00-05:00,0,12.0
...,...,...,...,...,...,...,...
1006,259860,356343,VEHICLE_TRANSPORTATION,2026-11-11 14:52:31.454000-05:00,2026-11-11 17:30:00-05:00,0,12.0
1064,260519,357039,VEHICLE_TRANSPORTATION,2026-11-11 14:54:25.816000-05:00,2026-11-11 17:30:00-05:00,0,12.0
1008,259878,356361,VEHICLE_TRANSPORTATION,2026-11-11 15:10:56.849000-05:00,2026-11-11 17:30:00-05:00,0,12.0
983,259683,356145,VEHICLE_TRANSPORTATION,2026-11-11 10:44:30.575000-05:00,2026-11-11 18:00:00-05:00,0,12.0


## Offline algorithm

In [8]:
labors = flexible_filter(labors_algo_df,
                city='149').sort_values(['schedule_date', 'created_at'])[['service_id', 'labor_id', 'labor_category', 'created_at', 'schedule_date',  'labor_sequence', 'assigned_driver', 'actual_start', 'actual_end', 'map_start_point', 'map_end_point']]

moves = flexible_filter(moves_algo_df,
                city='149').sort_values(['actual_start'])
print(f'Original services: {labors['service_id'].unique().size}')
print(f'Successful services: {moves['service_id'].unique().size}')
print('\n')
print(f'Original vt labors: {len(labors[labors['labor_category']=='VEHICLE_TRANSPORTATION'])}')
print(f'Successful vt labors: {len(moves[moves['labor_category'] == 'VEHICLE_TRANSPORTATION'])}')

Original services: 520
Successful services: 199


Original vt labors: 570
Successful vt labors: 210


In [9]:
print(f'Number of drivers: {len(flexible_filter(directorio_hist, city=149))}')
 

Number of drivers: 20


In [10]:
labors.groupby('assigned_driver').agg({'service_id':'count'}).reset_index()

,assigned_driver,service_id
0,10449,12
1,10451,12
2,10500,12
3,11712,12
4,11714,11
5,11988,10
6,14671,8
7,211,12
8,2331,12
9,26348,12


In [12]:
x = labors.drop_duplicates(subset=['service_id'])
x.groupby('schedule_date').agg({'service_id':'count'}).reset_index()

,schedule_date,service_id
0,2026-11-11 06:30:00-05:00,2
1,2026-11-11 07:00:00-05:00,44
2,2026-11-11 07:30:00-05:00,20
3,2026-11-11 07:31:00-05:00,1
4,2026-11-11 08:00:00-05:00,68
5,2026-11-11 08:01:00-05:00,1
6,2026-11-11 08:30:00-05:00,20
7,2026-11-11 09:00:00-05:00,11
8,2026-11-11 09:01:00-05:00,1
9,2026-11-11 09:30:00-05:00,12


# CASE 1: City 126

- Second and third driving labors are infeasible. First labor finishes after first one and second is 90km away from the first one. Third driving labor is on the next day.

## Instance

In [14]:
flexible_filter(
    labors_df,
    city=150
).sort_values(['schedule_date', 'created_at'])[['service_id', 'labor_id', 'labor_category', 'created_at', 'schedule_date',  'labor_sequence',  'labor_type']]

,service_id,labor_id,labor_category,created_at,schedule_date,labor_sequence,labor_type
827,257505,353824,VEHICLE_TRANSPORTATION,2026-11-06 09:55:43.798000-05:00,2026-11-11 08:00:00-05:00,0,12.0
1095,261108,357671,VEHICLE_TRANSPORTATION,2026-11-10 15:51:28.223000-05:00,2026-11-11 08:30:00-05:00,0,12.0
709,256396,352632,VEHICLE_TRANSPORTATION,2026-11-11 10:23:53.377000-05:00,2026-11-11 12:30:00-05:00,0,12.0
710,256396,352633,GENERAL_MECHANICS,2026-11-11 10:23:53.377000-05:00,2026-11-11 12:30:00-05:00,1,6.0
415,253078,349070,VEHICLE_TRANSPORTATION,2026-11-11 11:17:22.019000-05:00,2026-11-11 13:30:00-05:00,0,12.0
416,253078,349071,GENERAL_MECHANICS,2026-11-11 11:17:22.019000-05:00,2026-11-11 13:30:00-05:00,1,6.0
417,253078,351409,VEHICLE_TRANSPORTATION,2026-11-11 11:17:22.019000-05:00,2026-11-11 13:30:00-05:00,2,12.0


## Offline algorithm

In [22]:
flexible_filter(labors_algo_df,
                city='150').sort_values(['schedule_date', 'created_at'])[['service_id', 'labor_id', 'labor_category', 'created_at', 'schedule_date',  'labor_sequence', 'assigned_driver', 'actual_start', 'actual_end', 'map_start_point', 'map_end_point']]

,service_id,labor_id,labor_category,created_at,schedule_date,labor_sequence,assigned_driver,actual_start,actual_end,map_start_point,map_end_point
4,257505,353824,VEHICLE_TRANSPORTATION,2026-11-06 09:55:43.798000-05:00,2026-11-11 08:00:00-05:00,0,69147,2026-11-11 07:30:00-05:00,2026-11-11 10:34:44.200501-05:00,POINT (-75.5354366 10.4321803),POINT (-74.8751111 10.9621667)
5,261108,357671,VEHICLE_TRANSPORTATION,2026-11-10 15:51:28.223000-05:00,2026-11-11 08:30:00-05:00,0,<NA>,NaT,NaT,POINT (-75.4485626 10.3888702),POINT (-75.5200127 10.4018997)
3,256396,352632,VEHICLE_TRANSPORTATION,2026-11-11 10:23:53.377000-05:00,2026-11-11 12:30:00-05:00,0,<NA>,NaT,NaT,POINT (-75.516454 10.446315),POINT (-75.5308586 10.4214759)
0,253078,349070,VEHICLE_TRANSPORTATION,2026-11-11 11:17:22.019000-05:00,2026-11-11 13:30:00-05:00,0,69147,2026-11-11 13:35:50.919207-05:00,2026-11-11 14:25:37.083880-05:00,POINT (-75.516454 10.446315),POINT (-75.5308586 10.4214759)
1,253078,349071,GENERAL_MECHANICS,2026-11-11 11:17:22.019000-05:00,2026-11-11 13:30:00-05:00,1,<NA>,2026-11-11 14:25:37.083880-05:00,2026-11-12 05:50:44.941023-05:00,POINT (-75.5308586 10.4214759),POINT (-75.5308586 10.4214759)
2,253078,351409,VEHICLE_TRANSPORTATION,2026-11-11 11:17:22.019000-05:00,2026-11-11 13:30:00-05:00,2,<NA>,NaT,NaT,POINT (-75.5308586 10.4214759),POINT (-75.5051041 10.392139)


In [30]:
from src.data.distance_utils import distance
dist = distance('POINT (-74.8751111 10.9621667)', 'POINT (-75.516454 10.446315)', method='haversine')[0]
print(dist)
print(dist/ALFRED_SPEED*60)

90.55598922016596
181.1119784403319


In [19]:
flexible_filter(moves_algo_df,
                city='150').sort_values(['actual_start'])

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,city,duration_min,date
0,257505,353824,353824_free,FREE_TIME,FREE_TIME,69147,2026-11-11 08:00:00-05:00,2026-11-11 07:18:53.723085-05:00,2026-11-11 07:18:53.723085-05:00,POINT (-75.5051041 10.392139),POINT (-75.5051041 10.392139),0.000000,150,0.0,2026-11-11
1,257505,353824,353824_move,DRIVER_MOVE,DRIVER_MOVE,69147,2026-11-11 08:00:00-05:00,2026-11-11 07:18:53.723085-05:00,2026-11-11 07:30:00-05:00,POINT (-75.5051041 10.392139),POINT (-75.5354366 10.4321803),5.552308,150,11.1,2026-11-11
2,257505,353824,353824_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,2026-11-11 08:00:00-05:00,2026-11-11 07:30:00-05:00,2026-11-11 10:34:44.200501-05:00,POINT (-75.5354366 10.4321803),POINT (-74.8751111 10.9621667),93.157783,150,184.7,2026-11-11
3,253078,349070,349070_free,FREE_TIME,FREE_TIME,69147,2026-11-11 13:30:00-05:00,2026-11-11 10:34:44.200501-05:00,2026-11-11 10:34:44.200501-05:00,POINT (-74.8751111 10.9621667),POINT (-74.8751111 10.9621667),0.000000,150,0.0,2026-11-11
4,253078,349070,349070_move,DRIVER_MOVE,DRIVER_MOVE,69147,2026-11-11 13:30:00-05:00,2026-11-11 10:34:44.200501-05:00,2026-11-11 13:35:50.919207-05:00,POINT (-74.8751111 10.9621667),POINT (-75.516454 10.446315),90.555989,150,181.1,2026-11-11
5,253078,349070,349070_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,2026-11-11 13:30:00-05:00,2026-11-11 13:35:50.919207-05:00,2026-11-11 14:25:37.083880-05:00,POINT (-75.516454 10.446315),POINT (-75.5308586 10.4214759),3.179607,150,49.8,2026-11-11
